In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
training_data = pd.read_csv("data/training_data_utf8.csv")

/home/lorenz/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
val_data = pd.read_csv("data/validation_data_utf8.csv")

In [6]:
datum = val_data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")
share_neu = val_data.plot_area * val_data.plot_share

val_data["paidshare"] = share_neu
val_data["t_month"] = datum.dt.month
val_data["t_year"] = datum.dt.year

val_clean = val_data[["id", "price_per_m2","t_month","t_year","paidshare","property_type","cadastral","contract_type","protection_zone","building_ban"]]

In [7]:
datum = training_data.transaction_ym.apply(pd.to_datetime,format="%Y-%m")
share_neu = training_data.plot_area * training_data.plot_share

training_data["paidshare"] = share_neu
training_data["t_month"] = datum.dt.month
training_data["t_year"] = datum.dt.year

training_data_clean = training_data[["price_per_m2","t_month","t_year","paidshare","property_type","cadastral","contract_type","protection_zone","building_ban"]]
training_data_clean = training_data_clean.dropna()

In [8]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(training_data_clean, test_size=0.2, random_state=42)

In [9]:
data = train_set.drop("price_per_m2", axis=1)
labels = train_set["price_per_m2"].copy()

In [10]:
num_attribs = ["paidshare", "t_year", "t_month"]

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [12]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline

class LabelBinarizerPipelineFriendly(LabelBinarizer):
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelBinarizerPipelineFriendly, self).fit(X)
    def transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).fit(X).transform(X)

In [14]:
prop_pipeline = Pipeline([
    ("selector", DataFrameSelector("property_type")),
    ("label_binarizer", LabelBinarizerPipelineFriendly()),
])
cada_pipeline = Pipeline([
    ("selector", DataFrameSelector("cadastral")),
    ("label_binarizer", LabelBinarizerPipelineFriendly()),
])
cont_pipeline = Pipeline([
    ("selector", DataFrameSelector("contract_type")),
    ("label_binarizer", LabelBinarizerPipelineFriendly()),
])
prot_pipeline = Pipeline([
    ("selector", DataFrameSelector("protection_zone")),
    ("label_binarizer", LabelBinarizerPipelineFriendly()),
])
buil_pipeline = Pipeline([
    ("selector", DataFrameSelector("building_ban")),
    ("label_binarizer", LabelBinarizerPipelineFriendly()),
])
num_pipeline = Pipeline([
    ("selector", DataFrameSelector(num_attribs))
])

In [15]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("property_type_transform", prop_pipeline),
    ("cadastral_transform", cada_pipeline),
    ("contract_type_transform", cont_pipeline),
    ("protection_zone_transform", prot_pipeline),
    ("building_type", buil_pipeline)
])

In [16]:
data_prepared = full_pipeline.fit_transform(data)

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest_reg =  RandomForestRegressor()
forest_reg.fit(data_prepared, labels)

/home/lorenz/Projects/ML/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
scores = cross_val_score(forest_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

In [19]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())
          
display_scores(forest_rmse_scores)

Scores: [1087.36449437 2826.57960201 1719.91203808 3952.50175059 1227.11987126
 2238.81368246 2562.19823843 1503.41502159 3649.21316551 1928.16769444]
Mean: 2269.528555874026
Standard Deviation 928.5739844833809


In [20]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
lin_scores = cross_val_score(lin_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)

Scores: [1088.2915068  2610.87181706 1290.51609061 2626.97217855 1297.55595566
 2270.65086262 1866.21666916 1513.93304669 3503.24130927 1277.31737305]
Mean: 1934.5566809474526
Standard Deviation 752.3864169751598


In [22]:
test_data = test_set.drop("price_per_m2", axis=1)
test_labels = test_set["price_per_m2"].copy()
test_prepared = full_pipeline.transform(test_data)

In [23]:
from sklearn.metrics import mean_squared_error

predictlin = lin_reg.predict(test_prepared)
predictforest = forest_reg.predict(test_prepared)

lin_final_mse = mean_squared_error(test_labels, predictlin)
for_final_mse = mean_squared_error(test_labels, predictforest)

lin_rmse = np.sqrt(lin_final_mse)
forest_rmse = np.sqrt(for_final_mse)

print (lin_rmse, forest_rmse)

1219.7914318570308 1263.4651085140335


In [24]:
val_prepared = full_pipeline.transform(val_clean)

In [25]:
predictions = forest_reg.predict(val_prepared)

In [26]:
val_clean["res"] = predictions
val_clean[["id","res"]].set_index("id").to_csv("data/prediction.csv")

/home/lorenz/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
